In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

<a id= "1"> </a><br>
## Veri Setini Yüklüyoruz

In [ ]:
df = pd.read_csv("/kaggle/input/siber-zorbalk/tweetset.csv",encoding="windows-1254")
df.head()

Veri setinin içerisindeki özniteliklere bakacak olursak "Tip" ve "Paylaşım" olmak üzere 2 öznitelik vardır.

Bu özniteliklerin haricinde "NaN" yani boş değerlerle dolu olan öznitelikleri silmemiz gerekiyor .

**Paylaşım :** Sosyal medya platformu olan Tweetter'da kullanıcıların yaptıkları paylaşımların içeriklerini göstermektedir.

**Tip :** Atılan tweet lerin siber zorbalık ifade edip etmediğinin etiketlendiği öz niteliktir. "Pozitif","Negatif"

<a id= "2"> </a><br>
# Veri Ön İşleme

In [ ]:
# Veri setinde kayıp verilerin olup olmadığına bakıyoruz ve düzeltilemeyecek kadar olan feature'leri siliyoruz
print("Kayıp Veriler :{}".format(df.isnull().sum()))


In [ ]:
df.drop(["Unnamed: 2","Unnamed: 3","Unnamed: 4","Unnamed: 5"],axis=1,inplace=True)

In [ ]:
#Label encoder işlemi yaparak veri seti içerisinde bulunan "Negatif" değerli 0 "Pozitif" değerleri ise 1 yapıyoruz.
df["sınıf"] = [0 if (i=="Negatif") else 1 for i in df["Tip"]]
df.head()

In [ ]:
import string 
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

In [ ]:
#CLEANING SPECIAL CHARACTERS
def ozel_karakter (text):
    punctation = string.punctuation
    return text.translate(str.maketrans("","",punctation))

#CLEANING THE STOP WORDS FROM THE DATA SET
def stop_words_temizle (text):
    words = set(stopwords.words("turkish"))
    return (' '.join([i for i in text if i not in words and not i.isnumeric()]))


df["ozel_karakter"] = df["Paylaşım"].apply(lambda x: ozel_karakter(x))
df["ozel_karakter"] = df["ozel_karakter"].str.split()
df["stop_word"] = df["ozel_karakter"].apply(lambda x : stop_words_temizle(x))

In [ ]:
df['stop_word']= df['stop_word'].str.lower()
df.head()

In [ ]:
# VERİ SETİ İÇERİSİNDE İHTİYACIMIZ OLMAYAN SÜTUNLARI SİLİYORUZ
df.drop(["ozel_karakter","Paylaşım","Tip"],inplace=True,axis=1)
df.head()

In [ ]:

from sklearn.model_selection import train_test_split

X_train,X_test,Y_train,Y_test = train_test_split(df["stop_word"].values.astype('U'),
                                                 df["sınıf"],
                                                 test_size=0.2,
                                                 random_state=42)

In [ ]:
# COUNT VECTORİZE İLE SAYMA VEKTÖRLERİ OLUŞTURMA
from sklearn.feature_extraction.text import CountVectorizer

count_vect = CountVectorizer()
X_train_counts = count_vect.fit_transform(X_train)
print(X_train_counts.shape)

In [ ]:
# TFİIDF VEKTÖRÜ OLUŞTURMA SAYMA VEKTÖRLERİNİ TFIDF VEKTÖRLERİNE DÖNÜŞTÜRÜCEZ

from sklearn.feature_extraction.text import TfidfTransformer
tfidf =TfidfTransformer() 
X_train_tfidf= tfidf.fit_transform(X_train_counts)
print(X_train_tfidf.shape)

In [ ]:
# ÇOK MODLU NAIVE BAYES SINIFLANDIRICI EĞİTİYORUZ

from sklearn.naive_bayes import MultinomialNB

mnb = MultinomialNB()
clf = mnb.fit(X_train_tfidf, Y_train)

X_test_counts = count_vect.transform(X_test)
X_test_tfidf = tfidf.transform(X_test_counts)

In [ ]:
#Measuring Model Performance

y_pred = clf.predict(X_test_tfidf)

for text, sentiment in zip(X_test[:5],y_pred[:]):
    print("\n %r => %s" %(text,sentiment ))

In [ ]:
# Test Result
from sklearn.metrics import f1_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score


print(f"Accuracy Score %{round(accuracy_score(Y_test,y_pred)*100,2)}")
print(f"f-1 Score = %{round(f1_score(Y_test,y_pred)*100,2)}")
print(f"Precision = %{round(precision_score(Y_test,y_pred)*100,2)}")
print(f"Recall Score = %{round(recall_score(Y_test,y_pred)*100,2)}")
